In [2]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import io

# Provided SpaceX launch data as a string
csv_data = """Flight No.,FlightNumber,Date,Time,Launchoutcome,Outcome,BoosterVersion,Version Booster,Launch site,LaunchSite,Longitude,Latitude,Payload,PayloadMass,Orbit,Customer,Serial,Flights,GridFins,Reused,Legs,Block,ReusedCount,Booster landing,LandingPad
1,1,2006-03-24T22:30:00.000Z,22:30:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,FalconSAT-2,20.0,LEO,DARPA,Merlin1A,,False,False,False,,,No Attempt,
2,2,2007-03-21T01:10:00.000Z,01:10:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,DemoSAT,,LEO,DARPA,Merlin2A,,False,False,False,,,No Attempt,
3,3,2008-08-03T03:34:00.000Z,03:34:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,"Trailblazer, PRESat",,LEO,"NASA, ORS",Merlin1C,,False,False,False,,,No Attempt,
4,4,2008-09-28T23:15:00.000Z,23:15:00,True,True,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,RatSat,165.0,LEO,SpaceX,Merlin2C,,False,False,False,,,No Attempt,
5,5,2009-07-13T03:35:00.000Z,03:35:00,True,True,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,RazakSAT,200.0,LEO,ATSB,Merlin3C,,False,False,False,,,No Attempt,
6,6,2010-06-04T18:45:00.000Z,18:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Dragon Qualification Unit,,LEO,SpaceX,B0003,,False,False,False,,,No Attempt,
7,7,2010-12-08T15:43:00.000Z,15:43:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"COTS Demo Flight 1, Cubesats",,LEO,"NASA(COTS), NRO",B0004,,False,False,False,,,No Attempt,
8,8,2012-05-22T07:44:00.000Z,07:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,COTS Demo Flight 2,525.0,LEO,NASA(COTS),B0005,,False,False,False,,,No Attempt,
9,9,2012-10-08T00:35:00.000Z,00:35:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"CRS-1, Orbcomm-OG2",800.0,"ISS, LEO","NASA (CRS), Orbcomm",B0006,,False,False,False,,,No Attempt,
10,10,2013-03-01T19:10:00.000Z,19:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-2,677.0,ISS,NASA (CRS),B0007,,False,False,False,,,No Attempt,
11,11,2013-09-29T16:00:00.000Z,16:00:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,CASSIOPE,500.0,PO,MDA,B1003,,False,False,False,,,Failure,
12,12,2013-12-03T22:41:00.000Z,22:41:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-8,3170.0,GTO,SES,B1004,,False,False,False,,,No Attempt,
13,13,2014-01-06T18:06:00.000Z,18:06:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Thaicom 6,3325.0,GTO,Thaicom,B1005,,False,False,False,,,No Attempt,
14,14,2014-04-18T19:25:00.000Z,19:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-3,2296.0,ISS,NASA (CRS),B1006,,False,False,True,,,Success,
15,15,2014-07-14T15:15:00.000Z,15:15:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Orbcomm-OG2-M1,1316.0,LEO,Orbcomm,B1007,,False,False,True,,,Success,
16,16,2014-08-05T08:00:00.000Z,08:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,AsiaSat 8,4535.0,GTO,AsiaSat,B1008,,False,False,False,,,No Attempt,
17,17,2014-09-07T05:00:00.000Z,05:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,AsiaSat 6,4428.0,GTO,AsiaSat,B1011,,False,False,False,,,No Attempt,
18,18,2014-09-21T05:52:00.000Z,05:52:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-4,2216.0,ISS,NASA (CRS),B1010,,False,False,False,,,Failure,
19,19,2015-01-10T09:47:00.000Z,09:47:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-5,2395.0,ISS,NASA (CRS),B1012,,True,False,True,,,Failure,Just Read The Instructions V1
20,20,2015-02-11T23:03:00.000Z,23:03:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,DSCOVR,570.0,ES-L1,"NASA, NOAA, USAF",B1013,,True,False,True,,,Success,
21,21,2015-03-02T03:50:00.000Z,03:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"ABS-3A, Eutelsat 115 West B",4159.0,GTO,"ABS, Eutelsat",B1014,,False,False,False,,,No Attempt,
22,22,2015-04-14T20:10:00.000Z,20:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-6,1898.0,ISS,NASA (CRS),B1015,,True,False,True,,,Failure,Just Read The Instructions V1
23,23,2015-04-27T23:03:00.000Z,23:03:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,TürkmenÄlem 52°E / MonacoSAT,4707.0,GTO,Turkmenistan National Space Agency,B1016,,False,False,False,,,No Attempt,
24,24,2015-06-28T14:21:00.000Z,14:21:00,False,False,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-7,2477.0,ISS,NASA (CRS),B1018,,True,False,True,,,Failure,Of Course I Still Love You
25,25,2015-12-22T01:29:00.000Z,01:29:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Orbcomm-OG2-M2,2034.0,LEO,Orbcomm,B1019,,True,False,True,,,Success,Landing Zone 1
26,26,2016-01-17T15:42:00.000Z,15:42:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Jason-3,553.0,PO,"NASA, NOAA",B1017,,True,False,True,,,Failure,Just Read The Instructions
27,27,2016-03-04T23:35:00.000Z,23:35:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-9,5271.0,GTO,SES,B1020,,True,False,True,,,Failure,Of Course I Still Love You
28,28,2016-04-08T20:43:00.000Z,20:43:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-8,3136.0,ISS,NASA (CRS),B1021,,True,False,True,,,Success,Of Course I Still Love You
29,29,2016-05-06T05:21:00.000Z,05:21:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSAT-2B,4696.0,GTO,SKY Perfect JSAT Group,B1022,,True,False,True,,,Success,Of Course I Still Love You
30,30,2016-05-27T21:39:00.000Z,21:39:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Thaicom 8,3100.0,GTO,Thaicom,B1023,,True,False,True,,,Success,Of Course I Still Love You
31,31,2016-06-15T14:29:00.000Z,14:29:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"ABS-2A, Eutelsat 117 West B",3600.0,GTO,"ABS, Eutelsat",B1024,,True,False,True,,,Failure,Of Course I Still Love You
32,32,2016-07-18T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-9,2257.0,ISS,NASA (CRS),B1025,,True,False,True,,,Success,Landing Zone 1
33,33,2016-08-14T05:26:00.000Z,05:26:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSAT-16,4600.0,GTO,SKY Perfect JCSAT Group,B1026,,True,False,True,,,Success,Of Course I Still Love You
34,34,2016-09-01T13:07:00.000Z,13:07:00,False,False,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Amos-6,5500.0,GTO,Spacecom,B1028,,True,False,True,,,Failure,Of Course I Still Love You
35,35,2017-01-14T17:54:00.000Z,17:54:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 1,9600.0,PO,Iridium Communications,B1029,,True,False,True,,,Success,Just Read The Instructions
36,36,2017-02-19T14:39:00.000Z,14:39:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-10,2490.0,ISS,NASA (CRS),B1031,,True,False,True,,,Success,Landing Zone 1
37,37,2017-03-16T06:00:00.000Z,06:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,EchoStar 23,5600.0,GTO,EchoStar,B1030,,False,False,False,,,No Attempt,
38,38,2017-03-30T22:27:00.000Z,22:27:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,SES-10,5300.0,GTO,SES,B1021,,True,True,True,,,Success,Of Course I Still Love You
39,39,2017-05-01T11:15:00.000Z,11:15:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,NROL-76,,LEO,NRO,B1032,,True,False,True,,,Success,Landing Zone 1
40,40,2017-05-15T23:21:00.000Z,23:21:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Inmarsat-5 F4,6070.0,GTO,Inmarsat,B1034,,False,False,False,,,No Attempt,
41,41,2017-06-03T21:07:00.000Z,21:07:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-11,2708.0,ISS,NASA (CRS),B1035,,True,False,True,,,Success,Landing Zone 1
42,42,2017-06-23T19:10:00.000Z,19:10:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,BulgariaSat-1,3669.0,GTO,Bulgaria Sat,B1029,,True,True,True,,,Success,Of Course I Still Love You
43,43,2017-06-25T20:25:00.000Z,20:25:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 2,9600.0,PO,Iridium Communications,B1036,,True,False,True,,,Success,Just Read The Instructions
44,44,2017-07-05T23:35:00.000Z,23:35:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Intelsat 35e,6761.0,GTO,Intelsat,B1037,,False,False,False,,,No Attempt,
45,45,2017-08-14T16:31:00.000Z,16:31:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-12,2910.0,ISS,NASA (CRS),B1039,,True,False,True,,,Success,Landing Zone 1
46,46,2017-08-24T18:50:00.000Z,18:50:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,FormoSat-5,475.0,SSO,NSPO (Taiwan),B1038,,True,False,True,,,Success,Just Read The Instructions
47,47,2017-09-07T13:50:00.000Z,13:50:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,X-37B OTV-5,4990.0,LEO,USAF,B1040,,True,False,True,,,Success,Landing Zone 1
48,48,2017-10-09T12:37:00.000Z,12:37:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 3,9600.0,PO,Iridium Communications,B1041,,True,False,True,,,Success,Just Read The Instructions
49,49,2017-10-11T22:53:00.000Z,22:53:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,SES-11 / Echostar 105,5200.0,GTO,"Echostar, SES",B1031,,True,True,True,,,Success,Of Course I Still Love You
50,50,2017-10-30T19:34:00.000Z,19:34:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,KoreaSat 5A,3700.0,GTO,KT Corporation,B1042,,True,False,True,,,Success,Of Course I Still Love You
51,51,2017-12-15T15:36:00.000Z,15:36:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-13,2205.0,ISS,NASA (CRS),B1035,,True,True,True,,,Success,Landing Zone 1
52,52,2017-12-23T01:27:23.000Z,01:27:23,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 4,9600.0,PO,Iridium Communications,B1036,,True,True,False,,,Success,
53,53,2018-01-08T01:00:00.000Z,01:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,ZUMA,,LEO,Northrop Grumman,B1043,,True,False,True,,,Success,Landing Zone 1
54,54,2018-01-31T21:25:00.000Z,21:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GovSat-1,4230.0,GTO,GovSat,B1032,,True,True,True,,,Success,
55,55,2018-02-06T20:45:00.000Z,20:45:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Tesla Roadster,1350.0,HCO,SpaceX,B1033,,True,False,True,,,Failure,Of Course I Still Love You
56,56,2018-02-22T14:17:00.000Z,14:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,"Paz, Tintin A & B",2150.0,SSO,"HisdeSAT, SpaceX",B1038,,True,True,False,,,No Attempt,
57,57,2018-03-06T05:33:00.000Z,05:33:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Hispasat 30W-6,6092.0,GTO,Hispasat,B1044,,True,False,True,,,No Attempt,
58,58,2018-03-30T14:13:51.000Z,14:13:51,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 5,9600.0,PO,Iridium Communications,B1041,,True,True,True,,,No Attempt,
59,59,2018-04-02T20:30:41.000Z,20:30:41,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-14,2760.0,ISS,NASA (CRS),B1039,,True,True,True,,,No Attempt,
60,60,2018-04-18T22:51:00.000Z,22:51:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,TESS,350.0,HEO,NASA,B1045,,True,False,True,,,Success,Of Course I Still Love You
61,61,2018-05-11T20:14:00.000Z,20:14:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Bangabandhu-1,3750.0,GTO,Bangladesh Gov,B1046,,True,False,True,,,Success,Of Course I Still Love You
62,62,2018-05-22T19:47:58.000Z,19:47:58,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,"Iridium NEXT 6, GRACE-FO 1-2",5460.0,PO,"DLR, Iridium Communications, NASA",B1043,,True,True,False,,,No Attempt,
63,63,2018-06-04T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-12,5383.85,GTO,SES,B1040,,False,True,False,,,No Attempt,
64,64,2018-06-29T09:42:00.000Z,09:42:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-15,2410.0,ISS,NASA (CRS),B1045,,False,True,False,,,No Attempt,
65,65,2018-07-22T05:50:00.000Z,05:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telstar 19V,7076.0,GTO,Telesat,B1047,,True,False,True,,,Success,Of Course I Still Love You
66,66,2018-07-25T11:39:26.000Z,11:39:26,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 7,9600.0,PO,Iridium Communications,B1048,,True,False,True,,,Success,Just Read The Instructions
67,67,2018-08-07T05:18:00.000Z,05:18:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telkom-4,5800.0,GTO,Telkom,B1046,,True,True,True,,,Success,Of Course I Still Love You
68,68,2018-09-10T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telstar 18V,7060.0,GTO,Telesat,B1049,,True,False,True,,,Success,Of Course I Still Love You
69,69,2018-10-08T02:22:00.000Z,02:22:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SAOCOM 1A,2800.0,SSO,CONAE,B1048,,True,True,True,,,Success,Landing Zone 4
70,70,2018-11-15T20:46:00.000Z,20:46:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Es’hail 2,3000.0,GTO,Es’hailSat,B1047,,True,True,True,,,Success,Of Course I Still Love You
71,71,2018-12-03T18:34:00.000Z,18:34:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SSO-A,4000.0,SSO,"Spaceflight Industries, Inc",B1046,,True,True,True,,,Success,Just Read The Instructions
72,72,2018-12-05T18:16:00.000Z,18:16:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-16,2573.0,ISS,NASA (CRS),B1050,,True,False,True,,,Failure,Landing Zone 1
73,73,2018-12-23T13:51:00.000Z,13:51:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV01,4400.0,MEO,USAF,B1054,,False,False,False,,,No Attempt,
74,74,2019-01-11T15:31:00.000Z,15:31:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 8,9600.0,PO,Iridium Communications,B1049,,True,True,True,,,Success,Just Read The Instructions
75,75,2019-02-22T01:45:00.000Z,01:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"Nusantara Satu (PSN-6), S5, Beresheet",5645.0,GTO,"Air Force Research Laboratory, Pasifik Satelit Nusantara, SpaceIL",B1048,,True,True,True,,,Success,Of Course I Still Love You
76,76,2019-03-02T07:45:00.000Z,07:45:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CCtCap Demo Mission 1,12259.0,ISS,NASA (CCtCap),B1051,,True,False,True,,,Success,Of Course I Still Love You
77,77,2019-04-11T22:35:00.000Z,22:35:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,ArabSat 6A,6000.0,GTO,Arabsat,B1055,,True,False,True,,,Success,Of Course I Still Love You
78,78,2019-05-04T06:48:00.000Z,06:48:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-17,2482.0,ISS,NASA (CRS),B1056,,True,False,True,,,Success,Of Course I Still Love You
79,79,2019-05-24T02:30:00.000Z,02:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink v0.9,13200.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
80,80,2019-06-12T14:17:00.000Z,14:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,RADARSAT Constellation,1425.0,SSO,CSA,B1051,,True,True,True,,,Success,Landing Zone 4
81,81,2019-06-25T03:30:00.000Z,03:30:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"DSX, COSMIC-2, GPIM, Oculus-ASR, OTB 1, NPSAT 1, Prox-1, LightSail-B, ARMADILLO, FalconSat 7, E-TBEx A & B, PSat 2, BRICSat 2, TEPCE 1 & 2, CP9, StangSat",2838.0,"LEO, MEO","Cal Poly, General Atomics, Georgia Tech, Merrit Island HS, Michigan Tech, NASA, NOAA, Naval Postgrad School, Naval Research Lab, Planetary Society, SRI International, Taiwan, US Naval Academy, USAF, USAF Academy, University of Texas",B1057,,True,False,True,,,Failure,Of Course I Still Love You
82,82,2019-07-25T22:01:00.000Z,22:01:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-18,2227.7,ISS,NASA (CRS),B1056,,True,True,True,,,Success,Landing Zone 1
83,83,2019-08-06T22:52:00.000Z,22:52:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Amos-17,6500.0,GTO,Spacecom,B1047,,False,True,False,,,No Attempt,
84,84,2019-11-11T14:56:00.000Z,14:56:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-1,15600.0,VLEO,SpaceX,B1048,,True,True,True,,,Success,Of Course I Still Love You
85,85,2019-12-05T17:29:23.000Z,17:29:23,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-19,5000.0,ISS,NASA (CRS),B1059,,True,False,True,,,Success,Of Course I Still Love You
86,86,2019-12-17T00:10:00.000Z,00:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSat 18 / Kacific 1,6800.0,GTO,"Kacific Broadband Satellites Pte Ltd, SKY Perfect JSAT Corporation",B1056,,True,True,True,,,Success,Of Course I Still Love You
87,87,2020-01-07T02:19:00.000Z,02:19:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-2,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
88,88,2020-01-19T14:00:00.000Z,14:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew Dragon In Flight Abort Test,,SO,NASA (CCtCap),B1046,,False,True,False,,,No Attempt,
89,89,2020-01-29T14:06:00.000Z,14:06:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-3,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
90,90,2020-02-17T15:05:55.000Z,15:05:55,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-4,15600.0,VLEO,SpaceX,B1056,,True,True,True,,,Failure,Of Course I Still Love You
91,91,2020-03-07T04:50:31.000Z,04:50:31,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-20,1977.0,ISS,NASA (CRS),B1059,,True,True,True,,,Success,Landing Zone 1
92,92,2020-03-18T12:16:00.000Z,12:16:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-5,15600.0,VLEO,SpaceX,B1048,,True,True,True,,,Failure,Of Course I Still Love You
93,93,2020-04-22T19:30:00.000Z,19:30:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-6,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
94,94,2020-05-30T19:22:00.000Z,19:22:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CCtCap Demo Mission 2,9525.0,ISS,NASA (CCtCap),B1058,,True,False,True,,,Success,Of Course I Still Love You
95,95,2020-06-04T01:25:00.000Z,01:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-7,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Just Read The Instructions
96,96,2020-06-13T09:21:00.000Z,09:21:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-8,15600.0,VLEO,SpaceX,B1059,,True,True,True,,,Success,Of Course I Still Love You
97,97,2020-06-30T19:55:00.000Z,19:55:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV03 (Columbus),3880.0,MEO,USAF,B1060,,True,False,True,,,Success,Just Read The Instructions
98,98,2020-07-20T21:30:00.000Z,21:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,ANASIS-II,,GEO,South Korea,B1058,,True,True,True,,,Success,Just Read The Instructions
99,99,2020-08-07T05:12:00.000Z,05:12:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink-9, BlackSky Global 5-6",15712.0,"SSO, VLEO","BlackSky Global, SpaceX",B1051,,True,True,True,,,Success,Of Course I Still Love You
100,100,2020-08-18T14:31:00.000Z,14:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-10,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
101,101,2020-08-30T23:18:00.000Z,23:18:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SAOCOM 1B,1600.0,SSO,CONAE,B1059,,True,True,True,,,Success,Landing Zone 1
102,102,2020-09-03T12:46:00.000Z,12:46:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-11,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
103,103,2020-10-06T11:29:00.000Z,11:29:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-12,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Of Course I Still Love You
104,104,2020-10-18T12:25:00.000Z,12:25:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-13,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
105,105,2020-10-24T15:31:00.000Z,15:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-14,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
106,106,2020-11-05T23:24:00.000Z,23:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV04 (Sacagawea),3681.0,MEO,United States Space Force,B1062,,True,False,True,,,Success,Of Course I Still Love You
107,107,2020-11-16T00:27:00.000Z,00:27:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-1,,ISS,NASA (CCtCap),B1061,,True,False,True,,,Success,Just Read The Instructions
108,108,2020-11-21T17:17:00.000Z,17:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Sentinel-6 Michael Freilich,1440.0,PO,NASA,B1063,,True,False,True,,,Success,Landing Zone 4
109,109,2020-11-25T02:13:00.000Z,02:13:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-15,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
110,110,2020-12-06T16:17:00.000Z,16:17:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-21,2972.0,ISS,NASA (CRS),B1058,,True,True,True,,,Success,Of Course I Still Love You
111,111,2020-12-13T17:30:00.000Z,17:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SXM-7,7000.0,GTO,SiriusXM,B1051,,True,True,True,,,Success,Just Read The Instructions
112,112,2020-12-19T14:00:00.000Z,14:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,NROL-108,,,National Reconnaissance Office,B1059,,True,True,True,,,Success,Landing Zone 1
113,113,2021-01-08T02:15:00.000Z,02:15:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Turksat 5A,3500.0,GTO,Turksat,B1060,,True,True,True,,,Success,Just Read The Instructions
114,114,2021-01-20T13:02:00.000Z,13:02:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-16,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
115,115,2021-01-24T15:00:00.000Z,15:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-1,,SSO,,B1058,,True,True,True,,,Success,Of Course I Still Love You
116,116,2021-02-04T06:19:00.000Z,06:19:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-18,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
117,117,2021-02-16T03:59:00.000Z,03:59:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-19,15600.0,VLEO,SpaceX,B1059,,True,True,True,,,Failure,Of Course I Still Love You
118,118,2021-03-04T08:24:00.000Z,08:24:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-17,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
119,119,2021-03-11T08:13:00.000Z,08:13:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-20,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
120,120,2021-03-14T10:01:00.000Z,10:01:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-21,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
121,121,2021-03-24T08:28:00.000Z,08:28:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-22,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
122,122,2021-04-07T16:34:00.000Z,16:34:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-23,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Of Course I Still Love You
123,123,2021-04-23T09:49:00.000Z,09:49:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-2,,ISS,NASA (CCtCap),B1061,,True,True,True,,,Success,Of Course I Still Love You
124,124,2021-04-29T03:44:00.000Z,03:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-24,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
125,125,2021-05-04T19:01:00.000Z,19:01:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-25,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
126,126,2021-05-09T06:42:00.000Z,06:42:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-27,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
127,127,2021-05-15T22:54:00.000Z,22:54:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink-26, Capella-6, Tyvak-0130",13632.0,"LEO, VLEO","Capella Space, NOAA, SpaceX, Tyvak Nano-Satellite Systems Inc",B1058,,True,True,True,,,Success,Of Course I Still Love You
128,128,2021-05-26T18:59:00.000Z,18:59:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-28,15600.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Just Read The Instructions
129,129,2021-06-03T17:29:00.000Z,17:29:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-22,3328.0,ISS,NASA (CRS),B1067,,True,False,True,,,Success,Of Course I Still Love You
130,130,2021-06-06T04:26:00.000Z,04:26:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SXM-8,7000.0,GTO,SiriusXM,B1061,,True,True,True,,,Success,Just Read The Instructions
131,131,2021-06-17T16:09:00.000Z,16:09:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV05,3681.0,MEO,USAF,B1062,,True,True,True,,,Success,Just Read The Instructions
132,132,2021-06-30T19:31:00.000Z,19:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-2,,SSO,,B1060,,True,True,True,,,Success,Landing Zone 1
133,133,2021-08-29T07:14:00.000Z,07:14:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-23,,ISS,NASA (CRS),B1061,,True,True,True,,,Success,A Shortfall of Gravitas
134,134,2021-09-14T03:55:00.000Z,03:55:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 2-1 (v1.5),15600.0,PO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
135,135,2021-09-16T00:02:00.000Z,00:02:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Inspiration4,,LEO,Jared Isaacman,B1062,,True,True,True,,,Success,Just Read The Instructions
136,136,2021-11-11T02:03:00.000Z,11:03:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-3,12519.0,ISS,NASA (CCtCap),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
137,137,2021-11-13T12:40:00.000Z,12:40:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-1 (v1.5),13260.0,LEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
138,138,2021-11-24T06:20:00.000Z,06:20:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,DART (Double Asteroid Redirection Test),670.0,TLI,NASA,B1063,,True,True,True,,,Success,Of Course I Still Love You
139,139,2021-12-01T23:20:00.000Z,23:20:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-3 (v1.5),13260.0,PO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
140,140,2021-12-09T06:00:00.000Z,06:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,IPXE,330.0,LEO,"ASI, NASA",B1061,,True,True,True,,,Success,Just Read The Instructions
141,141,2021-12-18T12:41:40.000Z,12:41:40,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-4 (v1.5),13260.0,PO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
142,142,2021-12-19T03:58:00.000Z,03:58:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Türksat 5B,3500.0,GTO,Turksat,B1067,,True,True,True,,,Success,A Shortfall of Gravitas
143,143,2021-12-21T10:06:00.000Z,10:06:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-24,,ISS,NASA (CRS),B1069,,True,False,True,,,Success,Just Read The Instructions
144,144,2022-01-06T21:49:00.000Z,21:49:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-5 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
145,145,2022-01-13T15:25:00.000Z,15:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,NROL-87,,LEO,"National Reconnaissance Office, USAF",B1058,,True,True,True,,,Success,Landing Zone 1
146,146,2022-01-19T00:04:00.000Z,00:04:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-6 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
147,147,2022-01-31T23:11:12.000Z,23:11:12,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CSG-2,2240.0,SSO,ESA,B1052,,True,True,True,,,Success,Landing Zone 1
148,148,2022-02-02T20:18:00.000Z,20:18:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,NROL-87,,LEO,"National Reconnaissance Office, USAF",B1071,,True,False,True,,,Success,Landing Zone 4
149,149,2022-02-03T18:13:00.000Z,18:13:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-7 (v1.5),13260.0,VLEO,SpaceX,B1061,,True,True,True,,,Success,A Shortfall of Gravitas
150,150,2022-02-21T14:44:00.000Z,14:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-8 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,A Shortfall of Gravitas
151,151,2022-02-25T17:12:00.000Z,17:12:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-11 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
152,152,2022-03-03T14:35:00.000Z,14:35:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-9 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
153,153,2022-03-09T13:45:00.000Z,13:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-10 (v1.5),13260.0,VLEO,SpaceX,B1052,,True,True,True,,,Success,A Shortfall of Gravitas
154,154,2022-03-19T03:24:00.000Z,03:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-12 (v1.5),13260.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
155,155,2022-04-01T16:24:00.000Z,16:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-4,,SSO,,B1061,,True,True,True,,,Success,Just Read The Instructions
156,156,2022-04-08T15:17:00.000Z,15:17:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Ax-1,,ISS,Axiom Space,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
157,157,2022-04-17T13:13:00.000Z,13:13:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,NROL-85,,LEO,"National Reconnaissance Office, USAF",B1071,,True,True,True,,,Success,Landing Zone 4
158,158,2022-04-21T15:16:00.000Z,15:16:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-14 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
159,159,2022-04-27T07:52:00.000Z,07:52:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-4,,ISS,NASA (CCtCap),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
160,160,2022-04-29T21:27:00.000Z,21:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-16 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,Just Read The Instructions
161,161,2022-05-06T09:42:00.000Z,09:42:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-17 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,A Shortfall of Gravitas
162,162,2022-05-13T22:07:00.000Z,22:07:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-13 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
163,163,2022-05-14T20:40:00.000Z,20:40:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-15 (v1.5),13260.0,VLEO,SpaceX,B1073,,True,False,True,,,Success,Just Read The Instructions
164,164,2022-05-18T10:40:00.000Z,10:40:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-18 (v1.5),13260.0,VLEO,SpaceX,B1052,,True,True,True,,,Success,A Shortfall of Gravitas
165,165,2022-05-25T18:27:00.000Z,18:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-5,,SSO,,B1061,,True,True,True,,,Success,Landing Zone 1
166,166,2022-06-08T21:04:00.000Z,21:04:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Nilesat-301,3938.0,GEO,Nilesat,B1062,,True,True,True,,,Success,Just Read The Instructions
167,167,2022-06-01T17:08:50.000Z,17:08:50,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-19 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
168,168,2022-06-18T14:19:00.000Z,14:19:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SARah 1,2200.0,SSO,"AG, Bundeswehr, OHB-System",B1071,,True,True,True,,,Success,Landing Zone 4
169,169,2022-06-19T04:27:00.000Z,04:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Globalstar FM15,700.0,LEO,Globalstar,B1061,,True,True,True,,,Success,Just Read The Instructions
170,170,2022-06-29T21:04:00.000Z,21:04:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-22,3500.0,MEO,SES,B1073,,True,True,True,,,Success,A Shortfall of Gravitas
171,171,2022-07-07T13:11:00.000Z,13:11:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-21 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
172,172,2022-07-11T01:39:00.000Z,01:39:00,,,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-1 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
173,173,2022-07-15T00:44:00.000Z,00:44:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-25,,ISS,NASA (CRS),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
174,174,2022-07-17T14:50:00.000Z,14:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-22 (v1.5),13260.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
175,175,2022-07-21T17:13:00.000Z,17:13:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-2 (v1.5),13260.0,VLEO,SpaceX,B1071,,True,True,True,,,Success,Of Course I Still Love You
176,176,2022-07-24T00:00:00.000Z,00:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-25 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
177,177,2022-08-04T23:08:00.000Z,23:08:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,KPLO,678.0,TLI,KARI,B1052,,True,True,True,,,Success,Just Read The Instructions
178,178,2022-08-09T22:57:00.000Z,22:57:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-26 (v1.5),13260.0,VLEO,SpaceX,B1073,,True,True,True,,,Success,A Shortfall of Gravitas
179,179,2022-08-12T21:30:00.000Z,21:30:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-3 (v1.5),13260.0,VLEO,SpaceX,B1061,,True,True,True,,,Success,Of Course I Still Love You
180,180,2022-08-19T19:24:00.000Z,19:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-27 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
181,181,2022-08-28T02:22:00.000Z,02:22:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-23 (v1.5),13260.0,VLEO,SpaceX,B1069,,True,True,True,,,Success,A Shortfall of Gravitas
182,182,2022-08-31T05:40:00.000Z,05:40:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-4 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
183,183,2022-09-05T02:09:00.000Z,02:09:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"Starlink 4-20 (v1.5), Varuna-TDM",13440.0,VLEO,"Boeing, SpaceX",B1052,,True,True,True,,,Success,Just Read The Instructions
184,184,2022-09-11T01:10:00.000Z,01:10:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink 4-2 (v1.5), BlueWalker 3",14760.0,VLEO,"AST & Science, SpaceX",B1058,,True,True,True,,,Success,A Shortfall of Gravitas
185,185,2022-09-17T01:05:00.000Z,01:05:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-34 (v1.5),13260.0,VLEO,SpaceX,B1067,,True,True,True,,,Success,Just Read The Instructions
186,186,2022-09-24T23:30:00.000Z,23:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-35 (v1.5),13260.0,VLEO,SpaceX,B1072,,True,True,True,,,Success,Just Read The Instructions
187,187,2022-10-05T16:00:00.000Z,16:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-5,,ISS,NASA (CCtCap),B1077,,True,False,True,,,Success,Just Read The Instructions
"""

# Read the CSV data into a pandas DataFrame
df = pd.read_csv(io.StringIO(csv_data))

# Data Cleaning and Preparation
# Drop the potentially ambiguous 'Launch site' and 'Outcome' columns
# as 'LaunchSite' and 'Launchoutcome' (which will be renamed to 'LaunchOutcome')
# are the intended columns to use.
if 'Launch site' in df.columns:
    df.drop(columns=['Launch site'], inplace=True)
if 'Outcome' in df.columns:
    df.drop(columns=['Outcome'], inplace=True)

# Now rename 'Launchoutcome' for consistent camelCase naming
df.rename(columns={'Launchoutcome': 'LaunchOutcome'}, inplace=True)

# Convert 'LaunchOutcome' to boolean (True for success, False for failure)
df['LaunchOutcome'] = df['LaunchOutcome'].astype(bool)

# Group by Launch Site and calculate statistics
launch_site_stats = df.groupby('LaunchSite').agg(
    Latitude=('Latitude', 'first'),
    Longitude=('Longitude', 'first'),
    TotalLaunches=('LaunchOutcome', 'size'),
    SuccessfulLaunches=('LaunchOutcome', lambda x: x.sum())
).reset_index()

# Calculate Success Rate
launch_site_stats['SuccessRate'] = (launch_site_stats['SuccessfulLaunches'] / launch_site_stats['TotalLaunches'] * 100).fillna(0).round(2)

# Initialize the Folium map
# Start with a central location (e.g., average of all launch sites)
# If no data, default to a general SpaceX relevant location (Cape Canaveral)
if not launch_site_stats.empty:
    initial_latitude = launch_site_stats['Latitude'].mean()
    initial_longitude = launch_site_stats['Longitude'].mean()
else:
    initial_latitude = 28.5618571 # Cape Canaveral
    initial_longitude = -80.577366 # Cape Canaveral

spacex_map = folium.Map(location=[initial_latitude, initial_longitude], zoom_start=5)

# Add MarkerCluster to group nearby markers, improving readability
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add markers for each launch site to the map
for index, row in launch_site_stats.iterrows():
    site_name = row['LaunchSite']
    latitude = row['Latitude']
    longitude = row['Longitude']
    total_launches = row['TotalLaunches']
    successful_launches = row['SuccessfulLaunches']
    success_rate = row['SuccessRate']

    # Create a popup message for each marker
    popup_html = f"""
    <b>Launch Site:</b> {site_name}<br>
    <b>Total Launches:</b> {total_launches}<br>
    <b>Successful Launches:</b> {successful_launches}<br>
    <b>Success Rate:</b> {success_rate}%
    """

    # Add marker to the MarkerCluster
    folium.Marker(
        location=[latitude, longitude],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=site_name # Show site name on hover
    ).add_to(marker_cluster)

# Display the map (in Google Colab, this will render the map directly)
spacex_map


In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import io

# Provided SpaceX launch data as a string
csv_data = """Flight No.,FlightNumber,Date,Time,Launchoutcome,Outcome,BoosterVersion,Version Booster,Launch site,LaunchSite,Longitude,Latitude,Payload,PayloadMass,Orbit,Customer,Serial,Flights,GridFins,Reused,Legs,Block,ReusedCount,Booster landing,LandingPad
1,1,2006-03-24T22:30:00.000Z,22:30:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,FalconSAT-2,20.0,LEO,DARPA,Merlin1A,,False,False,False,,,No Attempt,
2,2,2007-03-21T01:10:00.000Z,01:10:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,DemoSAT,,LEO,DARPA,Merlin2A,,False,False,False,,,No Attempt,
3,3,2008-08-03T03:34:00.000Z,03:34:00,False,False,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,"Trailblazer, PRESat",,LEO,"NASA, ORS",Merlin1C,,False,False,False,,,No Attempt,
4,4,2008-09-28T23:15:00.000Z,23:15:00,True,True,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,RatSat,165.0,LEO,SpaceX,Merlin2C,,False,False,False,,,No Attempt,
5,5,2009-07-13T03:35:00.000Z,03:35:00,True,True,Falcon 1,Falcon 1,Kwajalein Atoll Omelek Island,Kwajalein Atoll Omelek Island,167.7431292,9.0477206,RazakSAT,200.0,LEO,ATSB,Merlin3C,,False,False,False,,,No Attempt,
6,6,2010-06-04T18:45:00.000Z,18:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Dragon Qualification Unit,,LEO,SpaceX,B0003,,False,False,False,,,No Attempt,
7,7,2010-12-08T15:43:00.000Z,15:43:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"COTS Demo Flight 1, Cubesats",,LEO,"NASA(COTS), NRO",B0004,,False,False,False,,,No Attempt,
8,8,2012-05-22T07:44:00.000Z,07:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,COTS Demo Flight 2,525.0,LEO,NASA(COTS),B0005,,False,False,False,,,No Attempt,
9,9,2012-10-08T00:35:00.000Z,00:35:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"CRS-1, Orbcomm-OG2",800.0,"ISS, LEO","NASA (CRS), Orbcomm",B0006,,False,False,False,,,No Attempt,
10,10,2013-03-01T19:10:00.000Z,19:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-2,677.0,ISS,NASA (CRS),B0007,,False,False,False,,,No Attempt,
11,11,2013-09-29T16:00:00.000Z,16:00:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,CASSIOPE,500.0,PO,MDA,B1003,,False,False,False,,,Failure,
12,12,2013-12-03T22:41:00.000Z,22:41:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-8,3170.0,GTO,SES,B1004,,False,False,False,,,No Attempt,
13,13,2014-01-06T18:06:00.000Z,18:06:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Thaicom 6,3325.0,GTO,Thaicom,B1005,,False,False,False,,,No Attempt,
14,14,2014-04-18T19:25:00.000Z,19:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-3,2296.0,ISS,NASA (CRS),B1006,,False,False,True,,,Success,
15,15,2014-07-14T15:15:00.000Z,15:15:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Orbcomm-OG2-M1,1316.0,LEO,Orbcomm,B1007,,False,False,True,,,Success,
16,16,2014-08-05T08:00:00.000Z,08:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,AsiaSat 8,4535.0,GTO,AsiaSat,B1008,,False,False,False,,,No Attempt,
17,17,2014-09-07T05:00:00.000Z,05:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,AsiaSat 6,4428.0,GTO,AsiaSat,B1011,,False,False,False,,,No Attempt,
18,18,2014-09-21T05:52:00.000Z,05:52:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-4,2216.0,ISS,NASA (CRS),B1010,,False,False,False,,,Failure,
19,19,2015-01-10T09:47:00.000Z,09:47:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-5,2395.0,ISS,NASA (CRS),B1012,,True,False,True,,,Failure,Just Read The Instructions V1
20,20,2015-02-11T23:03:00.000Z,23:03:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,DSCOVR,570.0,ES-L1,"NASA, NOAA, USAF",B1013,,True,False,True,,,Success,
21,21,2015-03-02T03:50:00.000Z,03:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"ABS-3A, Eutelsat 115 West B",4159.0,GTO,"ABS, Eutelsat",B1014,,False,False,False,,,No Attempt,
22,22,2015-04-14T20:10:00.000Z,20:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-6,1898.0,ISS,NASA (CRS),B1015,,True,False,True,,,Failure,Just Read The Instructions V1
23,23,2015-04-27T23:03:00.000Z,23:03:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,TürkmenÄlem 52°E / MonacoSAT,4707.0,GTO,Turkmenistan National Space Agency,B1016,,False,False,False,,,No Attempt,
24,24,2015-06-28T14:21:00.000Z,14:21:00,False,False,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-7,2477.0,ISS,NASA (CRS),B1018,,True,False,True,,,Failure,Of Course I Still Love You
25,25,2015-12-22T01:29:00.000Z,01:29:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Orbcomm-OG2-M2,2034.0,LEO,Orbcomm,B1019,,True,False,True,,,Success,Landing Zone 1
26,26,2016-01-17T15:42:00.000Z,15:42:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Jason-3,553.0,PO,"NASA, NOAA",B1017,,True,False,True,,,Failure,Just Read The Instructions
27,27,2016-03-04T23:35:00.000Z,23:35:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-9,5271.0,GTO,SES,B1020,,True,False,True,,,Failure,Of Course I Still Love You
28,28,2016-04-08T20:43:00.000Z,20:43:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-8,3136.0,ISS,NASA (CRS),B1021,,True,False,True,,,Success,Of Course I Still Love You
29,29,2016-05-06T05:21:00.000Z,05:21:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSAT-2B,4696.0,GTO,SKY Perfect JSAT Group,B1022,,True,False,True,,,Success,Of Course I Still Love You
30,30,2016-05-27T21:39:00.000Z,21:39:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Thaicom 8,3100.0,GTO,Thaicom,B1023,,True,False,True,,,Success,Of Course I Still Love You
31,31,2016-06-15T14:29:00.000Z,14:29:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"ABS-2A, Eutelsat 117 West B",3600.0,GTO,"ABS, Eutelsat",B1024,,True,False,True,,,Failure,Of Course I Still Love You
32,32,2016-07-18T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-9,2257.0,ISS,NASA (CRS),B1025,,True,False,True,,,Success,Landing Zone 1
33,33,2016-08-14T05:26:00.000Z,05:26:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSAT-16,4600.0,GTO,SKY Perfect JCSAT Group,B1026,,True,False,True,,,Success,Of Course I Still Love You
34,34,2016-09-01T13:07:00.000Z,13:07:00,False,False,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Amos-6,5500.0,GTO,Spacecom,B1028,,True,False,True,,,Failure,Of Course I Still Love You
35,35,2017-01-14T17:54:00.000Z,17:54:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 1,9600.0,PO,Iridium Communications,B1029,,True,False,True,,,Success,Just Read The Instructions
36,36,2017-02-19T14:39:00.000Z,14:39:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-10,2490.0,ISS,NASA (CRS),B1031,,True,False,True,,,Success,Landing Zone 1
37,37,2017-03-16T06:00:00.000Z,06:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,EchoStar 23,5600.0,GTO,EchoStar,B1030,,False,False,False,,,No Attempt,
38,38,2017-03-30T22:27:00.000Z,22:27:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,SES-10,5300.0,GTO,SES,B1021,,True,True,True,,,Success,Of Course I Still Love You
39,39,2017-05-01T11:15:00.000Z,11:15:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,NROL-76,,LEO,NRO,B1032,,True,False,True,,,Success,Landing Zone 1
40,40,2017-05-15T23:21:00.000Z,23:21:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Inmarsat-5 F4,6070.0,GTO,Inmarsat,B1034,,False,False,False,,,No Attempt,
41,41,2017-06-03T21:07:00.000Z,21:07:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-11,2708.0,ISS,NASA (CRS),B1035,,True,False,True,,,Success,Landing Zone 1
42,42,2017-06-23T19:10:00.000Z,19:10:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,BulgariaSat-1,3669.0,GTO,Bulgaria Sat,B1029,,True,True,True,,,Success,Of Course I Still Love You
43,43,2017-06-25T20:25:00.000Z,20:25:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 2,9600.0,PO,Iridium Communications,B1036,,True,False,True,,,Success,Just Read The Instructions
44,44,2017-07-05T23:35:00.000Z,23:35:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Intelsat 35e,6761.0,GTO,Intelsat,B1037,,False,False,False,,,No Attempt,
45,45,2017-08-14T16:31:00.000Z,16:31:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-12,2910.0,ISS,NASA (CRS),B1039,,True,False,True,,,Success,Landing Zone 1
46,46,2017-08-24T18:50:00.000Z,18:50:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,FormoSat-5,475.0,SSO,NSPO (Taiwan),B1038,,True,False,True,,,Success,Just Read The Instructions
47,47,2017-09-07T13:50:00.000Z,13:50:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,X-37B OTV-5,4990.0,LEO,USAF,B1040,,True,False,True,,,Success,Landing Zone 1
48,48,2017-10-09T12:37:00.000Z,12:37:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 3,9600.0,PO,Iridium Communications,B1041,,True,False,True,,,Success,Just Read The Instructions
49,49,2017-10-11T22:53:00.000Z,22:53:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,SES-11 / Echostar 105,5200.0,GTO,"Echostar, SES",B1031,,True,True,True,,,Success,Of Course I Still Love You
50,50,2017-10-30T19:34:00.000Z,19:34:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,KoreaSat 5A,3700.0,GTO,KT Corporation,B1042,,True,False,True,,,Success,Of Course I Still Love You
51,51,2017-12-15T15:36:00.000Z,15:36:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-13,2205.0,ISS,NASA (CRS),B1035,,True,True,True,,,Success,Landing Zone 1
52,52,2017-12-23T01:27:23.000Z,01:27:23,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 4,9600.0,PO,Iridium Communications,B1036,,True,True,False,,,Success,
53,53,2018-01-08T01:00:00.000Z,01:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,ZUMA,,LEO,Northrop Grumman,B1043,,True,False,True,,,Success,Landing Zone 1
54,54,2018-01-31T21:25:00.000Z,21:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GovSat-1,4230.0,GTO,GovSat,B1032,,True,True,True,,,Success,
55,55,2018-02-06T20:45:00.000Z,20:45:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Tesla Roadster,1350.0,HCO,SpaceX,B1033,,True,False,True,,,Failure,Of Course I Still Love You
56,56,2018-02-22T14:17:00.000Z,14:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,"Paz, Tintin A & B",2150.0,SSO,"HisdeSAT, SpaceX",B1038,,True,True,False,,,No Attempt,
57,57,2018-03-06T05:33:00.000Z,05:33:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Hispasat 30W-6,6092.0,GTO,Hispasat,B1044,,True,False,True,,,No Attempt,
58,58,2018-03-30T14:13:51.000Z,14:13:51,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 5,9600.0,PO,Iridium Communications,B1041,,True,True,True,,,No Attempt,
59,59,2018-04-02T20:30:41.000Z,20:30:41,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-14,2760.0,ISS,NASA (CRS),B1039,,True,True,True,,,No Attempt,
60,60,2018-04-18T22:51:00.000Z,22:51:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,TESS,350.0,HEO,NASA,B1045,,True,False,True,,,Success,Of Course I Still Love You
61,61,2018-05-11T20:14:00.000Z,20:14:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Bangabandhu-1,3750.0,GTO,Bangladesh Gov,B1046,,True,False,True,,,Success,Of Course I Still Love You
62,62,2018-05-22T19:47:58.000Z,19:47:58,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,"Iridium NEXT 6, GRACE-FO 1-2",5460.0,PO,"DLR, Iridium Communications, NASA",B1043,,True,True,False,,,No Attempt,
63,63,2018-06-04T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-12,5383.85,GTO,SES,B1040,,False,True,False,,,No Attempt,
64,64,2018-06-29T09:42:00.000Z,09:42:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-15,2410.0,ISS,NASA (CRS),B1045,,False,True,False,,,No Attempt,
65,65,2018-07-22T05:50:00.000Z,05:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telstar 19V,7076.0,GTO,Telesat,B1047,,True,False,True,,,Success,Of Course I Still Love You
66,66,2018-07-25T11:39:26.000Z,11:39:26,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 7,9600.0,PO,Iridium Communications,B1048,,True,False,True,,,Success,Just Read The Instructions
67,67,2018-08-07T05:18:00.000Z,05:18:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telkom-4,5800.0,GTO,Telkom,B1046,,True,True,True,,,Success,Of Course I Still Love You
68,68,2018-09-10T04:45:00.000Z,04:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Telstar 18V,7060.0,GTO,Telesat,B1049,,True,False,True,,,Success,Of Course I Still Love You
69,69,2018-10-08T02:22:00.000Z,02:22:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SAOCOM 1A,2800.0,SSO,CONAE,B1048,,True,True,True,,,Success,Landing Zone 4
70,70,2018-11-15T20:46:00.000Z,20:46:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Es’hail 2,3000.0,GTO,Es’hailSat,B1047,,True,True,True,,,Success,Of Course I Still Love You
71,71,2018-12-03T18:34:00.000Z,18:34:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SSO-A,4000.0,SSO,"Spaceflight Industries, Inc",B1046,,True,True,True,,,Success,Just Read The Instructions
72,72,2018-12-05T18:16:00.000Z,18:16:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-16,2573.0,ISS,NASA (CRS),B1050,,True,False,True,,,Failure,Landing Zone 1
73,73,2018-12-23T13:51:00.000Z,13:51:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV01,4400.0,MEO,USAF,B1054,,False,False,False,,,No Attempt,
74,74,2019-01-11T15:31:00.000Z,15:31:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Iridium NEXT 8,9600.0,PO,Iridium Communications,B1049,,True,True,True,,,Success,Just Read The Instructions
75,75,2019-02-22T01:45:00.000Z,01:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"Nusantara Satu (PSN-6), S5, Beresheet",5645.0,GTO,"Air Force Research Laboratory, Pasifik Satelit Nusantara, SpaceIL",B1048,,True,True,True,,,Success,Of Course I Still Love You
76,76,2019-03-02T07:45:00.000Z,07:45:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CCtCap Demo Mission 1,12259.0,ISS,NASA (CCtCap),B1051,,True,False,True,,,Success,Of Course I Still Love You
77,77,2019-04-11T22:35:00.000Z,22:35:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,ArabSat 6A,6000.0,GTO,Arabsat,B1055,,True,False,True,,,Success,Of Course I Still Love You
78,78,2019-05-04T06:48:00.000Z,06:48:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-17,2482.0,ISS,NASA (CRS),B1056,,True,False,True,,,Success,Of Course I Still Love You
79,79,2019-05-24T02:30:00.000Z,02:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink v0.9,13200.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
80,80,2019-06-12T14:17:00.000Z,14:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,RADARSAT Constellation,1425.0,SSO,CSA,B1051,,True,True,True,,,Success,Landing Zone 4
81,81,2019-06-25T03:30:00.000Z,03:30:00,True,True,Falcon Heavy,Falcon Heavy,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"DSX, COSMIC-2, GPIM, Oculus-ASR, OTB 1, NPSAT 1, Prox-1, LightSail-B, ARMADILLO, FalconSat 7, E-TBEx A & B, PSat 2, BRICSat 2, TEPCE 1 & 2, CP9, StangSat",2838.0,"LEO, MEO","Cal Poly, General Atomics, Georgia Tech, Merrit Island HS, Michigan Tech, NASA, NOAA, Naval Postgrad School, Naval Research Lab, Planetary Society, SRI International, Taiwan, US Naval Academy, USAF, USAF Academy, University of Texas",B1057,,True,False,True,,,Failure,Of Course I Still Love You
82,82,2019-07-25T22:01:00.000Z,22:01:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-18,2227.7,ISS,NASA (CRS),B1056,,True,True,True,,,Success,Landing Zone 1
83,83,2019-08-06T22:52:00.000Z,22:52:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Amos-17,6500.0,GTO,Spacecom,B1047,,False,True,False,,,No Attempt,
84,84,2019-11-11T14:56:00.000Z,14:56:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-1,15600.0,VLEO,SpaceX,B1048,,True,True,True,,,Success,Of Course I Still Love You
85,85,2019-12-05T17:29:23.000Z,17:29:23,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-19,5000.0,ISS,NASA (CRS),B1059,,True,False,True,,,Success,Of Course I Still Love You
86,86,2019-12-17T00:10:00.000Z,00:10:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,JCSat 18 / Kacific 1,6800.0,GTO,"Kacific Broadband Satellites Pte Ltd, SKY Perfect JSAT Corporation",B1056,,True,True,True,,,Success,Of Course I Still Love You
87,87,2020-01-07T02:19:00.000Z,02:19:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-2,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
88,88,2020-01-19T14:00:00.000Z,14:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew Dragon In Flight Abort Test,,SO,NASA (CCtCap),B1046,,False,True,False,,,No Attempt,
89,89,2020-01-29T14:06:00.000Z,14:06:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-3,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
90,90,2020-02-17T15:05:55.000Z,15:05:55,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-4,15600.0,VLEO,SpaceX,B1056,,True,True,True,,,Failure,Of Course I Still Love You
91,91,2020-03-07T04:50:31.000Z,04:50:31,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CRS-20,1977.0,ISS,NASA (CRS),B1059,,True,True,True,,,Success,Landing Zone 1
92,92,2020-03-18T12:16:00.000Z,12:16:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-5,15600.0,VLEO,SpaceX,B1048,,True,True,True,,,Failure,Of Course I Still Love You
93,93,2020-04-22T19:30:00.000Z,19:30:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-6,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
94,94,2020-05-30T19:22:00.000Z,19:22:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CCtCap Demo Mission 2,9525.0,ISS,NASA (CCtCap),B1058,,True,False,True,,,Success,Of Course I Still Love You
95,95,2020-06-04T01:25:00.000Z,01:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-7,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Just Read The Instructions
96,96,2020-06-13T09:21:00.000Z,09:21:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-8,15600.0,VLEO,SpaceX,B1059,,True,True,True,,,Success,Of Course I Still Love You
97,97,2020-06-30T19:55:00.000Z,19:55:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV03 (Columbus),3880.0,MEO,USAF,B1060,,True,False,True,,,Success,Just Read The Instructions
98,98,2020-07-20T21:30:00.000Z,21:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,ANASIS-II,,GEO,South Korea,B1058,,True,True,True,,,Success,Just Read The Instructions
99,99,2020-08-07T05:12:00.000Z,05:12:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink-9, BlackSky Global 5-6",15712.0,"SSO, VLEO","BlackSky Global, SpaceX",B1051,,True,True,True,,,Success,Of Course I Still Love You
100,100,2020-08-18T14:31:00.000Z,14:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-10,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
101,101,2020-08-30T23:18:00.000Z,23:18:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SAOCOM 1B,1600.0,SSO,CONAE,B1059,,True,True,True,,,Success,Landing Zone 1
102,102,2020-09-03T12:46:00.000Z,12:46:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-11,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
103,103,2020-10-06T11:29:00.000Z,11:29:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-12,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Of Course I Still Love You
104,104,2020-10-18T12:25:00.000Z,12:25:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-13,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
105,105,2020-10-24T15:31:00.000Z,15:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-14,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
106,106,2020-11-05T23:24:00.000Z,23:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV04 (Sacagawea),3681.0,MEO,United States Space Force,B1062,,True,False,True,,,Success,Of Course I Still Love You
107,107,2020-11-16T00:27:00.000Z,00:27:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-1,,ISS,NASA (CCtCap),B1061,,True,False,True,,,Success,Just Read The Instructions
108,108,2020-11-21T17:17:00.000Z,17:17:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Sentinel-6 Michael Freilich,1440.0,PO,NASA,B1063,,True,False,True,,,Success,Landing Zone 4
109,109,2020-11-25T02:13:00.000Z,02:13:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-15,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
110,110,2020-12-06T16:17:00.000Z,16:17:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-21,2972.0,ISS,NASA (CRS),B1058,,True,True,True,,,Success,Of Course I Still Love You
111,111,2020-12-13T17:30:00.000Z,17:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SXM-7,7000.0,GTO,SiriusXM,B1051,,True,True,True,,,Success,Just Read The Instructions
112,112,2020-12-19T14:00:00.000Z,14:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,NROL-108,,,National Reconnaissance Office,B1059,,True,True,True,,,Success,Landing Zone 1
113,113,2021-01-08T02:15:00.000Z,02:15:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Turksat 5A,3500.0,GTO,Turksat,B1060,,True,True,True,,,Success,Just Read The Instructions
114,114,2021-01-20T13:02:00.000Z,13:02:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-16,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
115,115,2021-01-24T15:00:00.000Z,15:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-1,,SSO,,B1058,,True,True,True,,,Success,Of Course I Still Love You
116,116,2021-02-04T06:19:00.000Z,06:19:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-18,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
117,117,2021-02-16T03:59:00.000Z,03:59:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-19,15600.0,VLEO,SpaceX,B1059,,True,True,True,,,Failure,Of Course I Still Love You
118,118,2021-03-04T08:24:00.000Z,08:24:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-17,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
119,119,2021-03-11T08:13:00.000Z,08:13:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-20,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
120,120,2021-03-14T10:01:00.000Z,10:01:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-21,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
121,121,2021-03-24T08:28:00.000Z,08:28:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-22,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Of Course I Still Love You
122,122,2021-04-07T16:34:00.000Z,16:34:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-23,15600.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Of Course I Still Love You
123,123,2021-04-23T09:49:00.000Z,09:49:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-2,,ISS,NASA (CCtCap),B1061,,True,True,True,,,Success,Of Course I Still Love You
124,124,2021-04-29T03:44:00.000Z,03:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-24,15600.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
125,125,2021-05-04T19:01:00.000Z,19:01:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink-25,15600.0,VLEO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
126,126,2021-05-09T06:42:00.000Z,06:42:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-27,15600.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
127,127,2021-05-15T22:54:00.000Z,22:54:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink-26, Capella-6, Tyvak-0130",13632.0,"LEO, VLEO","Capella Space, NOAA, SpaceX, Tyvak Nano-Satellite Systems Inc",B1058,,True,True,True,,,Success,Of Course I Still Love You
128,128,2021-05-26T18:59:00.000Z,18:59:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink-28,15600.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Just Read The Instructions
129,129,2021-06-03T17:29:00.000Z,17:29:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-22,3328.0,ISS,NASA (CRS),B1067,,True,False,True,,,Success,Of Course I Still Love You
130,130,2021-06-06T04:26:00.000Z,04:26:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SXM-8,7000.0,GTO,SiriusXM,B1061,,True,True,True,,,Success,Just Read The Instructions
131,131,2021-06-17T16:09:00.000Z,16:09:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,GPS III SV05,3681.0,MEO,USAF,B1062,,True,True,True,,,Success,Just Read The Instructions
132,132,2021-06-30T19:31:00.000Z,19:31:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-2,,SSO,,B1060,,True,True,True,,,Success,Landing Zone 1
133,133,2021-08-29T07:14:00.000Z,07:14:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-23,,ISS,NASA (CRS),B1061,,True,True,True,,,Success,A Shortfall of Gravitas
134,134,2021-09-14T03:55:00.000Z,03:55:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 2-1 (v1.5),15600.0,PO,SpaceX,B1049,,True,True,True,,,Success,Of Course I Still Love You
135,135,2021-09-16T00:02:00.000Z,00:02:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Inspiration4,,LEO,Jared Isaacman,B1062,,True,True,True,,,Success,Just Read The Instructions
136,136,2021-11-11T02:03:00.000Z,11:03:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-3,12519.0,ISS,NASA (CCtCap),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
137,137,2021-11-13T12:40:00.000Z,12:40:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-1 (v1.5),13260.0,LEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
138,138,2021-11-24T06:20:00.000Z,06:20:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,DART (Double Asteroid Redirection Test),670.0,TLI,NASA,B1063,,True,True,True,,,Success,Of Course I Still Love You
139,139,2021-12-01T23:20:00.000Z,23:20:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-3 (v1.5),13260.0,PO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
140,140,2021-12-09T06:00:00.000Z,06:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,IPXE,330.0,LEO,"ASI, NASA",B1061,,True,True,True,,,Success,Just Read The Instructions
141,141,2021-12-18T12:41:40.000Z,12:41:40,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-4 (v1.5),13260.0,PO,SpaceX,B1051,,True,True,True,,,Success,Of Course I Still Love You
142,142,2021-12-19T03:58:00.000Z,03:58:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Türksat 5B,3500.0,GTO,Turksat,B1067,,True,True,True,,,Success,A Shortfall of Gravitas
143,143,2021-12-21T10:06:00.000Z,10:06:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-24,,ISS,NASA (CRS),B1069,,True,False,True,,,Success,Just Read The Instructions
144,144,2022-01-06T21:49:00.000Z,21:49:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-5 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
145,145,2022-01-13T15:25:00.000Z,15:25:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,NROL-87,,LEO,"National Reconnaissance Office, USAF",B1058,,True,True,True,,,Success,Landing Zone 1
146,146,2022-01-19T00:04:00.000Z,00:04:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-6 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
147,147,2022-01-31T23:11:12.000Z,23:11:12,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,CSG-2,2240.0,SSO,ESA,B1052,,True,True,True,,,Success,Landing Zone 1
148,148,2022-02-02T20:18:00.000Z,20:18:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,NROL-87,,LEO,"National Reconnaissance Office, USAF",B1071,,True,False,True,,,Success,Landing Zone 4
149,149,2022-02-03T18:13:00.000Z,18:13:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-7 (v1.5),13260.0,VLEO,SpaceX,B1061,,True,True,True,,,Success,A Shortfall of Gravitas
150,150,2022-02-21T14:44:00.000Z,14:44:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-8 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,A Shortfall of Gravitas
151,151,2022-02-25T17:12:00.000Z,17:12:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-11 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
152,152,2022-03-03T14:35:00.000Z,14:35:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-9 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
153,153,2022-03-09T13:45:00.000Z,13:45:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-10 (v1.5),13260.0,VLEO,SpaceX,B1052,,True,True,True,,,Success,A Shortfall of Gravitas
154,154,2022-03-19T03:24:00.000Z,03:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-12 (v1.5),13260.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
155,155,2022-04-01T16:24:00.000Z,16:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-4,,SSO,,B1061,,True,True,True,,,Success,Just Read The Instructions
156,156,2022-04-08T15:17:00.000Z,15:17:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Ax-1,,ISS,Axiom Space,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
157,157,2022-04-17T13:13:00.000Z,13:13:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,NROL-85,,LEO,"National Reconnaissance Office, USAF",B1071,,True,True,True,,,Success,Landing Zone 4
158,158,2022-04-21T15:16:00.000Z,15:16:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-14 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,Just Read The Instructions
159,159,2022-04-27T07:52:00.000Z,07:52:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-4,,ISS,NASA (CCtCap),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
160,160,2022-04-29T21:27:00.000Z,21:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-16 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,Just Read The Instructions
161,161,2022-05-06T09:42:00.000Z,09:42:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-17 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,A Shortfall of Gravitas
162,162,2022-05-13T22:07:00.000Z,22:07:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 4-13 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
163,163,2022-05-14T20:40:00.000Z,20:40:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-15 (v1.5),13260.0,VLEO,SpaceX,B1073,,True,False,True,,,Success,Just Read The Instructions
164,164,2022-05-18T10:40:00.000Z,10:40:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-18 (v1.5),13260.0,VLEO,SpaceX,B1052,,True,True,True,,,Success,A Shortfall of Gravitas
165,165,2022-05-25T18:27:00.000Z,18:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Transporter-5,,SSO,,B1061,,True,True,True,,,Success,Landing Zone 1
166,166,2022-06-08T21:04:00.000Z,21:04:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Nilesat-301,3938.0,GEO,Nilesat,B1062,,True,True,True,,,Success,Just Read The Instructions
167,167,2022-06-01T17:08:50.000Z,17:08:50,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-19 (v1.5),13260.0,VLEO,SpaceX,B1060,,True,True,True,,,Success,A Shortfall of Gravitas
168,168,2022-06-18T14:19:00.000Z,14:19:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,SARah 1,2200.0,SSO,"AG, Bundeswehr, OHB-System",B1071,,True,True,True,,,Success,Landing Zone 4
169,169,2022-06-19T04:27:00.000Z,04:27:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Globalstar FM15,700.0,LEO,Globalstar,B1061,,True,True,True,,,Success,Just Read The Instructions
170,170,2022-06-29T21:04:00.000Z,21:04:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,SES-22,3500.0,MEO,SES,B1073,,True,True,True,,,Success,A Shortfall of Gravitas
171,171,2022-07-07T13:11:00.000Z,13:11:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-21 (v1.5),13260.0,VLEO,SpaceX,B1058,,True,True,True,,,Success,Just Read The Instructions
172,172,2022-07-11T01:39:00.000Z,01:39:00,,,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-1 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
173,173,2022-07-15T00:44:00.000Z,00:44:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,CRS-25,,ISS,NASA (CRS),B1067,,True,True,True,,,Success,A Shortfall of Gravitas
174,174,2022-07-17T14:50:00.000Z,14:50:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-22 (v1.5),13260.0,VLEO,SpaceX,B1051,,True,True,True,,,Success,Just Read The Instructions
175,175,2022-07-21T17:13:00.000Z,17:13:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-2 (v1.5),13260.0,VLEO,SpaceX,B1071,,True,True,True,,,Success,Of Course I Still Love You
176,176,2022-07-24T00:00:00.000Z,00:00:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-25 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
177,177,2022-08-04T23:08:00.000Z,23:08:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,KPLO,678.0,TLI,KARI,B1052,,True,True,True,,,Success,Just Read The Instructions
178,178,2022-08-09T22:57:00.000Z,22:57:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-26 (v1.5),13260.0,VLEO,SpaceX,B1073,,True,True,True,,,Success,A Shortfall of Gravitas
179,179,2022-08-12T21:30:00.000Z,21:30:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-3 (v1.5),13260.0,VLEO,SpaceX,B1061,,True,True,True,,,Success,Of Course I Still Love You
180,180,2022-08-19T19:24:00.000Z,19:24:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-27 (v1.5),13260.0,VLEO,SpaceX,B1062,,True,True,True,,,Success,A Shortfall of Gravitas
181,181,2022-08-28T02:22:00.000Z,02:22:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Starlink 4-23 (v1.5),13260.0,VLEO,SpaceX,B1069,,True,True,True,,,Success,A Shortfall of Gravitas
182,182,2022-08-31T05:40:00.000Z,05:40:00,True,True,Falcon 9,Falcon 9,Vandenberg Space Force Base Space Launch Complex 4E,Vandenberg Space Force Base Space Launch Complex 4E,-120.610829,34.632093,Starlink 3-4 (v1.5),13260.0,VLEO,SpaceX,B1063,,True,True,True,,,Success,Of Course I Still Love You
183,183,2022-09-05T02:09:00.000Z,02:09:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,"Starlink 4-20 (v1.5), Varuna-TDM",13440.0,VLEO,"Boeing, SpaceX",B1052,,True,True,True,,,Success,Just Read The Instructions
184,184,2022-09-11T01:10:00.000Z,01:10:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,"Starlink 4-2 (v1.5), BlueWalker 3",14760.0,VLEO,"AST & Science, SpaceX",B1058,,True,True,True,,,Success,A Shortfall of Gravitas
185,185,2022-09-17T01:05:00.000Z,01:05:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-34 (v1.5),13260.0,VLEO,SpaceX,B1067,,True,True,True,,,Success,Just Read The Instructions
186,186,2022-09-24T23:30:00.000Z,23:30:00,True,True,Falcon 9,Falcon 9,Cape Canaveral Space Force Station Space Launch Complex 40,Cape Canaveral Space Force Station Space Launch Complex 40,-80.577366,28.5618571,Starlink 4-35 (v1.5),13260.0,VLEO,SpaceX,B1072,,True,True,True,,,Success,Just Read The Instructions
187,187,2022-10-05T16:00:00.000Z,16:00:00,True,True,Falcon 9,Falcon 9,Kennedy Space Center Historic Launch Complex 39A,Kennedy Space Center Historic Launch Complex 39A,-80.6039558,28.6080585,Crew-5,,ISS,NASA (CCtCap),B1077,,True,False,True,,,Success,Just Read The Instructions
"""

# Read the CSV data into a pandas DataFrame
df = pd.read_csv(io.StringIO(csv_data))

# Data Cleaning and Preparation
# Drop the potentially ambiguous 'Launch site' and 'Outcome' columns
# as 'LaunchSite' and 'Launchoutcome' (which will be renamed to 'LaunchOutcome')
# are the intended columns to use.
if 'Launch site' in df.columns:
    df.drop(columns=['Launch site'], inplace=True)
if 'Outcome' in df.columns:
    df.drop(columns=['Outcome'], inplace=True)

# Now rename 'Launchoutcome' for consistent camelCase naming
df.rename(columns={'Launchoutcome': 'LaunchOutcome'}, inplace=True)

# Convert 'LaunchOutcome' to boolean (True for success, False for failure)
df['LaunchOutcome'] = df['LaunchOutcome'].astype(bool)

# Group by Launch Site and calculate statistics
launch_site_stats = df.groupby('LaunchSite').agg(
    Latitude=('Latitude', 'first'),
    Longitude=('Longitude', 'first'),
    TotalLaunches=('LaunchOutcome', 'size'),
    SuccessfulLaunches=('LaunchOutcome', lambda x: x.sum())
).reset_index()

# Calculate Success Rate
launch_site_stats['SuccessRate'] = (launch_site_stats['SuccessfulLaunches'] / launch_site_stats['TotalLaunches'] * 100).fillna(0).round(2)

# Initialize the Folium map
# Start with a central location (e.g., average of all launch sites)
# If no data, default to a general SpaceX relevant location (Cape Canaveral)
if not launch_site_stats.empty:
    initial_latitude = launch_site_stats['Latitude'].mean()
    initial_longitude = launch_site_stats['Longitude'].mean()
else:
    initial_latitude = 28.5618571 # Cape Canaveral
    initial_longitude = -80.577366 # Cape Canaveral

spacex_map = folium.Map(location=[initial_latitude, initial_longitude], zoom_start=5)

# Add MarkerCluster to group nearby markers, improving readability
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add markers for each launch site to the map
for index, row in launch_site_stats.iterrows():
    site_name = row['LaunchSite']
    latitude = row['Latitude']
    longitude = row['Longitude']
    total_launches = row['TotalLaunches']
    successful_launches = row['SuccessfulLaunches']
    success_rate = row['SuccessRate']

    # Create a popup message for each marker
    popup_html = f"""
    <b>Launch Site:</b> {site_name}<br>
    <b>Total Launches:</b> {total_launches}<br>
    <b>Successful Launches:</b> {successful_launches}<br>
    <b>Success Rate:</b> {success_rate}%
    """

    # Add marker to the MarkerCluster
    folium.Marker(
        location=[latitude, longitude],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=site_name # Show site name on hover
    ).add_to(marker_cluster)

# Display the map (in Google Colab, this will render the map directly)
spacex_map


In [ ]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians
import os # For saving the HTML file

# Define the calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculates the distance between two points on Earth using the Haversine formula.
    Args:
        lat1, lon1: Latitude and longitude of the first point.
        lat2, lon2: Latitude and longitude of the second point.
    Returns:
        The distance in kilometers.
    """
    R = 6373.0  # approximate radius of earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# --- Data Loading and Preparation ---
# Directly read the spacex_launch_geo.csv dataset from the Colab environment
csv_file_path = 'spacex_launch_geo.csv'
try:
    spacex_df = pd.read_csv(csv_file_path)
    print(f"Dataset '{csv_file_path}' loaded successfully from Colab directory.")
except FileNotFoundError:
    print(f"Error: '{csv_file_path}' not found in the current Colab directory.")
    print(f"Please upload 'spacex_launch_geo.csv' to your Colab session storage.")
    print(f"You can download it from: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv")
    spacex_df = pd.DataFrame() # Create an empty DataFrame to prevent further errors

if spacex_df.empty:
    print("DataFrame is empty. Map generation aborted.")
else:
    # Select relevant sub-columns and filter for 'KSC LC-39A'
    spacex_df_filtered = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
    ksc_lc_39a_df = spacex_df_filtered[spacex_df_filtered['Launch Site'] == 'KSC LC-39A'].copy()

    # Get the coordinates for KSC LC-39A
    if not ksc_lc_39a_df.empty:
        launch_site_lat = ksc_lc_39a_df['Lat'].iloc[0]
        launch_site_lon = ksc_lc_39a_df['Long'].iloc[0]
        ksc_lc_39a_coords = [launch_site_lat, launch_site_lon]
    else:
        print("KSC LC-39A data not found in the dataset. Using default coordinates.")
        ksc_lc_39a_coords = [28.573255, -80.646895] # Default if not found in data
        launch_site_lat, launch_site_lon = ksc_lc_39a_coords

    # Initialize the map centered at KSC LC-39A with an appropriate zoom level
    ksc_map = folium.Map(location=ksc_lc_39a_coords, zoom_start=15)

    # --- TASK 1: Mark launch site on the map ---
    # Add a highlighted circle area for the launch site
    folium.Circle(
        ksc_lc_39a_coords,
        radius=100, # meters
        color='#000000', # Black border
        fill=True,
        fill_color='#d35400' # Orange fill
    ).add_child(folium.Popup('KSC LC-39A Launch Site')).add_to(ksc_map)

    # Add a marker with a DivIcon for the launch site name
    folium.map.Marker(
        ksc_lc_39a_coords,
        icon=DivIcon(
            icon_size=(200,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12pt; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A'
        )
    ).add_to(ksc_map)

    # --- TASK 2: Mark success/failed launches for KSC LC-39A on the map ---
    # Function to assign color based on launch outcome (1 for success, 0 for failure)
    def assign_marker_color(launch_outcome):
        return 'green' if launch_outcome == 1 else 'red'

    # Create a new 'marker_color' column based on the 'class' column
    ksc_lc_39a_df['marker_color'] = ksc_lc_39a_df['class'].apply(assign_marker_color)

    # Create a MarkerCluster object to group overlapping markers
    marker_cluster = MarkerCluster()
    ksc_map.add_child(marker_cluster)

    # Add individual markers for each launch record to the marker cluster
    for index, record in ksc_lc_39a_df.iterrows():
        folium.Marker(
            [record['Lat'], record['Long']],
            icon=folium.Icon(color='white', icon_color=record['marker_color'])
        ).add_to(marker_cluster)

    # --- TASK 3: Calculate the distances between KSC LC-39A to its proximities ---
    # Add Mouse Position plugin to get coordinates on mouse hover
    formatter = "function(num) {return L.Util.formatNum(num, 5);};"
    mouse_position = MousePosition(
        position='topright',
        separator=' Long: ',
        empty_string='NaN',
        lng_first=False,
        num_digits=20,
        prefix='Lat:',
        lat_formatter=formatter,
        lng_formatter=formatter,
    )
    ksc_map.add_child(mouse_position)

    # Approximate coordinates for a coastline, railway, and highway near KSC LC-39A
    # These are illustrative coordinates; you can use the MousePosition tool on the map
    # to find more precise locations for your points of interest.
    coastline_coords = [28.573255, -80.5700] # East towards the Atlantic Ocean
    railway_coords = [28.5720, -80.6550]   # West of launch site, on a railway track
    highway_coords = [28.5720, -80.6800]   # West of launch site, on State Road 3

    # Calculate distances from KSC LC-39A to these proximity points
    distance_coast = calculate_distance(launch_site_lat, launch_site_lon, coastline_coords[0], coastline_coords[1])
    distance_rail = calculate_distance(launch_site_lat, launch_site_lon, railway_coords[0], railway_coords[1])
    distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_coords[0], highway_coords[1])

    # Create and add markers for the calculated distances with descriptive labels
    folium.Marker(
        coastline_coords,
        icon=DivIcon(
            icon_size=(250, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:blue;"><b>Coastline: {distance_coast:.2f} km</b></div>'
        )
    ).add_to(ksc_map)

    folium.Marker(
        railway_coords,
        icon=DivIcon(
            icon_size=(250, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:orange;"><b>Railway: {distance_rail:.2f} km</b></div>'
        )
    ).add_to(ksc_map)

    folium.Marker(
        highway_coords,
        icon=DivIcon(
            icon_size=(250, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:purple;"><b>Highway: {distance_highway:.2f} km</b></div>'
        )
    ).add_to(ksc_map)

    # Draw PolyLines connecting the launch site to the proximity points
    ksc_lc_39a_site_coords = [launch_site_lat, launch_site_lon]

    folium.PolyLine(
        locations=[ksc_lc_39a_site_coords, coastline_coords],
        weight=2, # Line thickness
        color='blue',
        popup=f'Distance to Coastline: {distance_coast:.2f} km'
    ).add_to(ksc_map)

    folium.PolyLine(
        locations=[ksc_lc_39a_site_coords, railway_coords],
        weight=2,
        color='orange',
        popup=f'Distance to Railway: {distance_rail:.2f} km'
    ).add_to(ksc_map)

    folium.PolyLine(
        locations=[ksc_lc_39a_site_coords, highway_coords],
        weight=2,
        color='purple',
        popup=f'Distance to Highway: {distance_highway:.2f} km'
    ).add_to(ksc_map)

    # Save the map to an HTML file
    map_filename = 'ksc_lc_39a_proximity_map.html'
    ksc_map.save(map_filename)

    print(f"Folium map for 'KSC Launch Complex 39A Proximity Map' saved to {map_filename}")

    # For Google Colab, you can also display the map directly in the notebook:
    # from IPython.display import HTML
    # HTML(ksc_map._repr_html_())